# DEMO: Diffusion prompts Generator

This notebook is a demonstrator of the generated prompts. It uses code from the `llm/prompts_generator.py`.

### Libraries

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import json
import random

In [6]:
import sys
sys.path.append('..')
from llm.prompts_generator import load_model

### Model and tokenizer loader

In [7]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"
model, tokenizer = load_model(model_name_or_path)


### Load class list as input of the prompts

In [8]:
# Load class list
with open('../llm/class_list.json', 'r') as file:
    class_list = json.load(file)

### Set seed for reproducibility
42 is moslty used as seed number and considered as the lucky number. 

In [9]:
seed = 42
random.seed(seed)

### Let's configuration the LLM model for the prompts generation

In [10]:
# Initialize the prompt template and tokenizer
prompt = ""
prompt_template=f'''{prompt}'''

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Same as the "self-rewarding language models" paper
generation_params = {
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_new_tokens": 512,
    "repetition_penalty": 1.1
}

# Using the text streamer to stream output one token at a time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# SYSTEM_PROMPT to instruct the model on specificities
SYSTEM_PROMPT = "<s>[SYS] Generate concise prompts for a text-to-image diffusion generator using stable diffusion. Focus on the specified class give specific details of the scene and avoid additional information, notes, suggestions and discussions.[/SYS]"


### Random selection of 4 classes for the test

In [11]:
# Shuffle the classes to pick them randomly
class_keys = list(class_list.keys())
random.shuffle(class_keys)

selected_classes_keys = random.sample(class_keys, 4)

# Get the class names based on the selected keys
selected_classes = [class_list[class_key] for class_key in selected_classes_keys]

selected_classes

['hair drier', 'truck', 'toaster', 'cup']

5 prompts generation per selected class

In [12]:
for class_id in selected_classes_keys:
    class_name = class_list[class_id]

    # Combine SYSTEM_PROMPT and class-specific prompt
    prompt = f"{SYSTEM_PROMPT}<s>[INST] Give 5 distinct, simple and detailed scene prompts for a text-to-image diffusion generator using stable diffusion with a {class_name} [/INST]"

    tokens = tokenizer(
        prompt,
        return_tensors='pt'
    ).input_ids.cuda()

    print('\n' + class_name + '\n')
    generation_output = model.generate(
        tokens,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer,
        **generation_params
    )


hair drier

1. A close-up image of an old, vintage hairdryer with a chrome body, a white button in the center, and a faded blue power cord coiled neatly at the bottom.

2. A modern hairdrier with a sleek black design, a digital display screen showing temperature settings, and a long pink extension cord reaching towards an outlet.

3. An image of a hand holding a small, compact travel hairdryer with a mint green exterior, two buttons, and a short white plug attached to it.

4. A hairdryer with a unique shape – resembling a pinecone – in a bathroom setting, surrounded by steamy water droplets on a white tile floor, a towel hanging nearby, and a mirror reflecting the scene.

5. A professional hair salon scene: A hairdresser holding a large, powerful hairdryer with multiple heat and speed settings, blowing hot air onto a client's freshly washed and conditioned hair while they sit in a styling chair, looking relaxed with their eyes closed.

truck

1. A red firetruck parked in front of a bu